In [2]:
import os as os
import sys as sys
import re as re
import pandas as pd
import numpy as np
import json as json
import pickle
import urllib as urllib
import zlib as zlib
import base64 as base64
from requests import Request, Session
import requests
from numpy import trapz
import itertools
import zlib as zlib
import base64 as base64
import pickle
import pathlib
from tqdm import tqdm
from datetime import datetime
import time
import copy
from Bio import SeqIO
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.dummy import DummyClassifier

pd.options.display.max_columns = 100
pd.options.display.min_rows = None
pd.options.display.max_rows = 20
pd.options.display.max_colwidth = 100

DATA_PATH = pathlib.Path('/data1/home/adpatter/gene-to-phenotype-predictions/adpatter/data/')
PROTEIN_SEQUENCE_PATH = DATA_PATH.joinpath('protein_sequences_effect_size_sample.pkl')

In [3]:
df = pd.read_pickle(PROTEIN_SEQUENCE_PATH)

df

,gene_symbol_harmonized,est_m_ea,seq
0,0610009B22Rik,-0.56979,MSGSFYFVIVGHHDNPVFEMEFLPAGKAESKDEHRHLNQFIAHAALDLVDENMWLSNNMYLKTVDKFNEWFVSAFVTAGHMRLIMLHDVRHEDGIK...
1,0610040J01Rik,-0.956256,MGCRCCKMIQSYLFDPVQVPSPGFVNEVNNCKLEEDDTVRLKGTQNSEVEVPRNALHDGSLSNSESRGSTTGLPHQGPLPQEDSEERPCVEKQGIV...
2,1110017D15Rik,-0.136771,MFLFSRKTKTPISTYSDSYRAPTSIKEVYKDPPLWAWEANKFVTPGLTQTMHRHVDPEALQKMTKCAAQDYTYKSSISGHPYLPEKYWLSPDEEDK...
3,1110032A03Rik,0.221307,MAVSCSLNHSTYLQRQNLVCYLRNPHYGSLIYADGHGEVWTDWNDMSKFLQYGWRCTTNENSYSNRTLFGHAFETTYDANYSRKKPQSTHRFKREP...
4,1110059G10Rik,0.016842,CRVLFIVGQNQNYHEQAEPSVLCAARRARLPVPLQRESRLQGGAHRRDQDEEPPPADGRIVYRKPVKRSSDEKCSGLTASSKKKKTNEDDVNKQSS...
5,1500009L16Rik,-1.00767,XPTGAVKDTTEDSITEDDKRRNYGGVYVGLPSEAVNMASSQTKTVQKKEDNMTAGYSRACS
6,1500011B03Rik,-0.208127,MLRLGWMRLLPVLCSLLLGRAEAPSPGLPPEQSRSYEVLRRQSLDLSNYLKGNWSQELDRLHKELQIQIQKLNRTRVEPVTLGGFTSWLTSAFSYF...
7,1600014C10Rik,-0.011545,MPIMVDDIMRLLCSISQERKMKAAVKHSGKGAMVAGAMAFVGGLVGGPPGIAVGGTVGGLLGAWMTSGQFKPVPQILMELPPR
8,1600029I14Rik,1.432683,MHFSSSETIGSFTLYQPVSGLSTAQKCLLMLCMEAGTERTPTESWRCSSPVSSSQIKTRKLHRVLRLKLWWAPWRTLACLSVSEDWNAVLSSAGIK...
9,1700001O22Rik,0.523172,MFRRQSMEGTPAYKPKGYPDGTTQRRPVKRLPMLLPMLEGRVGRDVDGAWRRDRKSQRLPCGSPESPNARRDRGTARALLLPSLQPSSSAREAGTR...


In [ ]:
df = df_data.copy()

print(df.shape)

X = df['seq'].tolist()
y = df['est_m_ea'].apply(lambda x: 0 if x <= 0 else 1).tolist()

In [ ]:
param_grid = {
    'tfidfVectorizer__ngram_range': [(42, 42), (2000,2000)],
    'tfidfVectorizer__norm': ('l1', 'l2'),
    'logisticRegression__C': [0.001,0.01,0.1,1,10,100, 1000],
}

pipe = Pipeline(steps=[
    ('tfidfVectorizer',  TfidfVectorizer(analyzer='char')), 
    ('logisticRegression', LogisticRegression())
    ])

search = GridSearchCV(estimator=pipe, param_grid=param_grid, n_jobs=2)
search.fit(X, y)

In [ ]:
df = pd.DataFrame(search.cv_results_)

df

In [ ]:
param_grid = {
    'tfidfVectorizer__ngram_range': [(42, 42), (2000,2000)],
    'tfidfVectorizer__norm': ('l1', 'l2')
}

pipe = Pipeline(steps=[
    ('tfidfVectorizer',  TfidfVectorizer(analyzer='char')), 
    ('dummyClassifier', DummyClassifier(strategy='most_frequent'))
    ])

dummySearch = GridSearchCV(estimator=pipe, param_grid=param_grid, n_jobs=2)
dummySearch.fit(X, y)

In [ ]:
df = pd.DataFrame(dummySearch.cv_results_)

df